In [1]:

%pip install qdrant-client
%pip install cohere langchain qdrant-client tfds-nightly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from IPython.display import display, Markdown, Latex
import os


In [3]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Now you can get the API key from the environment
api_key = os.getenv('COHERE_API_KEY')

# Use the API key
os.environ["COHERE_API_KEY"] = api_key


In [4]:
# save the embeddings in a DB that is persistent
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Directory containing the PDFs
pdf_directory = 'data'  # Replace with your PDFs folder path

# Function to read and split a single PDF into chunks
def process_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', ' '],  # Adjust separators as needed
        chunk_size=500,  # Adjust this size according to your needs
        chunk_overlap=100,  # Adjust overlap as needed
        length_function=len,
        add_start_index=True,
    )
    return text_splitter.split_documents(documents)

# Function to process all PDFs in the directory and return text chunks and metadata
def process_all_pdfs(directory):
    all_chunks = []
    all_metadata = []
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory, filename)
            chunks = process_pdf(pdf_path)
            for chunk in chunks:
                all_chunks.append(chunk.page_content)
                # Here you can customize the metadata as needed
                all_metadata.append({"source": filename, "chunk_start": chunk.metadata['start_index']})
    return all_chunks, all_metadata

# Process all PDFs and get the text chunks and their metadata
texts, metadata = process_all_pdfs(pdf_directory)

# Now, you have a list of text chunks in `texts` and their corresponding metadata in `metadata`
# You can now proceed to embed these chunks and store them

# Define the embeddings model
embeddings = CohereEmbeddings(model="embed-english-v3.0")

# Load the .env file
load_dotenv()

# Now you can get the API key from the environment
URL = os.getenv('URL')
API_KEY = os.getenv('QDRANT-API-KEY')


# Embed the documents and store in the vector store (use an actual path for persistent storage)
vector_store = Qdrant.from_texts(texts, embeddings, url = URL, api_key=API_KEY, prefer_grpc=True, metadatas=metadata, collection_name="pdfs", distance_func="Dot")

# Now you can use the retriever as before
retriever = vector_store.as_retriever(search_kwargs={"k": 30})


In [5]:
retriever=vector_store.as_retriever()

#create custom prompt for your use case
prompt_template="""You are an expert in O Level Physics. Your task is to give accurate abnd precise answrs to any and all questions that students ask you.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""


messages = [
    SystemMessagePromptTemplate.from_template(prompt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

chain_type_kwargs = {"prompt": prompt}

llm=Cohere(model="command-nightly")


chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def print_result(result):
    # Retrieve the actual content of the source documents and format them as markdown list
    source_contents = '\n\n'.join([f"- {doc.page_content}" for doc in result['source_documents']])
    relevant_sources = ', '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))
    
    output_text = f"""
### Question:
{query}

### Answer:
{result['answer']}

### Sources:
{source_contents}

### All relevant sources:
{relevant_sources}
"""
    return output_text.strip()  # Using strip to remove any leading/trailing whitespace



In [6]:
query = "What are waves?"
result = chain(query)
display(Markdown(print_result(result)))

c:\Users\dt\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


### Question:
What are waves?

### Answer:
 Waves are a phenomenon where energy is transferred through vibrations. This occurs by creating oscillations in a medium that transfers these vibrations to other points in the medium, creating the wave effect. The medium can be anything from a physical substance like water or air, to a field such as electromagnetic fields, like light. One key aspect of waves is that the transfer of energy is not necessarily accompanied by a transfer of matter.

 Waves can be formed in various ways, such as moving a rope up and down rapidly, creating ripples in a ripple tank, or using a slinky spring. These demonstrations help visualize the concept of waves, illustrating that waves are formed through a back-and-forth motion that creates oscillations between two extreme positions.

The most important property of waves is that they do not necessarily propagate in a straight line. Rather, they can take various paths, depending on the properties of the medium they propagate through. This makes waves a complex but essential natural phenomenon that plays a crucial role in many scientific fields, from physics to engineering. 

There are two different types of wave motion, transverse waves and longitudinal waves. Transverse waves are waves that travel perpendicular to the direction of motion, like the waves created by shaking a rope. Longitudinal waves, on

### Sources:
- a.A wave is a phenomenon where energy is 
transferred through vibrations.
b.One complete motion from one extreme position 
to another and back is known as an 
oscillation/vibration.
c.Waves can be formed by moving a rope up and 
down, using a ripple tank or using a slinky spring.
d.The source of any wave is a vibration or 
oscillation. 
2.There are 2 types of wave motion: Transverse and 
Longitudinal.
1.Transverse waves are waves that travel perpendicularto the direction of motion.

- can be used to make objects look bigger.  
b. Camera  
c. LCD Projector  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Physics Chapter 13: Waves  
Chapter 13.1: Describing Waves  
 
1. A wave is made up of periodic motion , which is 
motion repeated at regular intervals . 
a. A wave is a phenomenon where energy is 
transferred through vibrations . 
b. One complete motion from one extreme position 
to another and back is known as an 
oscillation/vibrati on.

- 10. Echo is reflected  sound from hard and smooth surfaces.  
11. Sound is produced  by vibrations  placed  in a medium 
12. A Wave is a phenomenon  in which energy is transferred  through vibrations  
 
13. Transverse  waves are waves where the direction  of travel is perpendicular  to the 
direction  of vibration  of particle in a medium.        (Light/water  are transverse  waves) 
 
14. Longitudinal  waves are waves where the direction  of travel is parallel to the direction

- (e.g. sound waves).  
2. The following words are used to describe waves:  
• Amplitude  – the distance from the horizontal axis to the peak (in m).  
• Wavelength ( l) – the distance from peak to peak, or trough to  trough (in m).  
• Frequency – the number of complete waves per second (in Hz).  
• Period – the time taken for one complete wavelength (in s).  
3. All waves can be reflected, refracted and diffracted:  
• Reflection – a wave bouncing off a surface.

### All relevant sources:
physicswavesrevision-100809074420-phpapp02.pdf, physics general v2-02 (revised).pdf, physics waves revision.pdf, physics_waves.pdf

In [7]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [8]:
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
#
compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

a.A wave is a phenomenon where energy is 
transferred through vibrations.
b.One complete motion from one extreme position 
to another and back is known as an 
oscillation/vibration.
c.Waves can be formed by moving a rope up and 
down, using a ripple tank or using a slinky spring.
d.The source of any wave is a vibration or 
oscillation. 
2.There are 2 types of wave motion: Transverse and 
Longitudinal.
1.Transverse waves are waves that travel perpendicularto the direction of motion.
----------------------------------------------------------------------------------------------------
Document 2:

10. Echo is reflected  sound from hard and smooth surfaces.  
11. Sound is produced  by vibrations  placed  in a medium 
12. A Wave is a phenomenon  in which energy is transferred  through vibrations  
 
13. Transverse  waves are waves where the direction  of travel is perpendicular  to the 
direction  of vibration  of particle in a medium.        (Light/water  are transverse  waves)

In [9]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [10]:
query = "What are waves?"
result = chain(query)
display(Markdown(print_result(result)))

### Question:
What are waves?

### Answer:
 Waves are a phenomenon where energy is transferred through vibrations. It is a periodic motion that is repeated at regular intervals, where one complete motion from one extreme position to another and back is known as an oscillation or vibration. Waves can be formed by moving a rope up and down, using a ripple tank, or using a slinky spring. 

 Waves transfer energy efficiently, and the speed of the wave is affected by the medium that it travels through. The denser the medium, the slower the wave, and as the density decreases, the speed of the wave increases. 

There are two types of wave motions: 

1. Transverse waves travel perpendicularly to the direction of motion 
2. Longitudinal waves travel parallel to the direction of motion. 

Some examples of waves include water waves, sound waves, seismic waves, gravity waves, and electromagnetic waves. 

Would you like me to go into more detail on a specific area of wave coverage? 

### Sources:
- a.A wave is a phenomenon where energy is 
transferred through vibrations.
b.One complete motion from one extreme position 
to another and back is known as an 
oscillation/vibration.
c.Waves can be formed by moving a rope up and 
down, using a ripple tank or using a slinky spring.
d.The source of any wave is a vibration or 
oscillation. 
2.There are 2 types of wave motion: Transverse and 
Longitudinal.
1.Transverse waves are waves that travel perpendicularto the direction of motion.

- 10. Echo is reflected  sound from hard and smooth surfaces.  
11. Sound is produced  by vibrations  placed  in a medium 
12. A Wave is a phenomenon  in which energy is transferred  through vibrations  
 
13. Transverse  waves are waves where the direction  of travel is perpendicular  to the 
direction  of vibration  of particle in a medium.        (Light/water  are transverse  waves) 
 
14. Longitudinal  waves are waves where the direction  of travel is parallel to the direction

- can be used to make objects look bigger.  
b. Camera  
c. LCD Projector  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Physics Chapter 13: Waves  
Chapter 13.1: Describing Waves  
 
1. A wave is made up of periodic motion , which is 
motion repeated at regular intervals . 
a. A wave is a phenomenon where energy is 
transferred through vibrations . 
b. One complete motion from one extreme position 
to another and back is known as an 
oscillation/vibrati on.

### All relevant sources:
physicswavesrevision-100809074420-phpapp02.pdf, physics general v2-02 (revised).pdf, physics waves revision.pdf